# TOC
- <a href="#Data-reading">Data Reading</a>
    - <a href="#Read-Testing">Read Testing</a>
- <a href="#Finding-Indels">Finding Indels</a>
- <a href="#Finding-Differences">Finding Differences</a>
- <a href="#Replacement-Codons">Replacement Codons</a>
- <a href="#Codon-Swap">Codon Swap</a>
- <a href="#Build-Primers">Build Primers</a>
- <a href="#REALIZATIONS">REALIZATIONS</a>
- <a href="#Table-Generation">Codon Table Generation</a>

# Data reading

<a href="#TOC">TOC</a>

In [1]:
from Bio import SeqIO
from Bio.Data import CodonTable
from Bio.Seq import Seq
from pandas import DataFrame

In [149]:
def read_clustal(filepath, data_dict):
    """
    Reads a clustal protein alignment file. Raises an exception if more than
    two sequences are in the alignment.
    
    Parameters
    ----------
    filepath: str
        path to the clustal file
    
    data_dict: dict
        The data dictionary to be populated. Defined above.
        
    TODO
    ----
    Add a try/except to check that the path and file exists. Import os stuff.
    """
    
    for i, s in enumerate(SeqIO.parse(filename, format='clustal')):
        try:
            assert i < 2
        except:
            raise Exception("More than two sequences.")
        data['protein{}'.format(i+1)]['id'].append(s.id)
        data['protein{}'.format(i+1)]['seq_string'] = str(s.seq)
        data['protein{}'.format(i+1)]['description'].append(s.description)
        data['protein{}'.format(i+1)]['name'].append(s.name)
        #data['protein{}'.format(i+1)]['seq_list'] = parse_seq_align(data['protein{}'.format(i+1)]['seq_string'])
        data['protein{}'.format(i+1)]['seq_list'] = list(str(s.seq))

Why do I need `parse_seq_align`?

In [150]:
def parse_seq_align(seq_string):
    """
    Parses an alignment string that may have indels present.
    
    Parameters
    ----------
    seq_string: str
        Protein sequence from a clustal alignment file.
        
    Returns
    -------
    seq_list: list
        Protein sequence as a list with indels removed.
    """
    
    return list(''.join(seq_string.split('-')))

In [151]:
def read_dna(filename, data):
    """
    Reads a fasta file that contains a single DNA sequence. Raises an exception if 
    more than one sequence is in the file.
    
    Parameters
    ----------
    filepath: str
        path to the fasta file
    
    data_dict: dict
        The data dictionary to be populated. Defined above.
        
    TODO
    ----
    Add a try/except to check that the path and file exists. Import os stuff.

    """

    
    for i, s in enumerate(SeqIO.parse(filename, format='fasta')):
        try:
            assert i < 1
        except:
            raise Exception("More than one sequence in file.")
        data['dna{}'.format(i+1)]['id'].append(s.id)
        data['dna{}'.format(i+1)]['seq_string'] = str(s.seq)
        data['dna{}'.format(i+1)]['description'].append(s.description)
        data['dna{}'.format(i+1)]['name'].append(s.name)
        s = data['dna1']['seq_string'] 
        codon_list = [s[i:i+3].upper() for i in range(0, len(s), 3)]
        data['dna{}'.format(i+1)]['codon_list'] = codon_list
        t = CodonTable.standard_dna_table
        data['dna{}'.format(i+1)]['translated_list'] = [t.forward_table[i] for i in codon_list]
    

## Read Testing  
<a href="#TOC">TOC</a>

In [296]:
data = {
    'protein1': {
                 'id': [], 
                 'seq_string':[],
                 'seq_list':[],
                 'reduced_list':[],
                 'description': [], 
                 'name':[]
                },
    'protein2': {
                 'id': [], 
                 'seq_string':[],
                 'seq_list':[],
                 'reduced_list':[],
                 'description': [], 
                 'name':[]
                },
    'dna1': {
             'id': [], 
             'seq_string':[], 
             'description': [], 
             'name':[], 
             'codon_list':[], 
             'translated_list':[]
            },
    'diff_dict': {
                  'swap_index': [], 
                  'swap_old_aa': [], 
                  'swap_new_aa': [], 
                  'swap_new_codon': [],
                  'indel_index': [],
                  'indel_new_aa': [],
                  'indel_new_codon': []                 
                 },
    'meta': {'time':'', 
            }
}


In [298]:
filename = 'mouse_opossum_alignment-20171115.clustal'
read_clustal(filename, data)
filename = 'mouse_ly96_dna.fasta'
read_dna(filename, data)

# Finding Indels
<a href="#TOC">TOC</a>

## Solution
I'm going to split up finding differences into two groups:
1. Finding insertion sites
2. Substitutions and deletions

Deletions pose a weird problem because I lose the index number that I need to change the codon. This issue isn't present for deletions since the original sequence still has that codon.

In [275]:
data.keys()

dict_keys(['protein1', 'protein2', 'dna1', 'diff_dict', 'meta'])

In [299]:
def find_indels(data):
    """
    Finds indices and replacement amino acids for indels in the 
    template protein sequence.
    
    Parameters
    ----------
    data: dict
        A dictionary defined previously that contains protein 
        sequence lists.
        
    Returns
    -------
    Nothing. Populates the data dictionary with reduced protein
    sequences, the indel list, and insertion list.
    """
    
    # Copying new instances of the protein lists
    temp_prot1_list = data['protein1']['seq_list'][:]
    temp_prot2_list = data['protein2']['seq_list'][:]
    
    # Getting a list of indels in the first sequence
    indel_list = [i for i,res in enumerate(temp_prot1_list) if res=='-']
    
    # Getting the inserted residue at the indel postions
    insert_list = [data['protein2']['seq_list'][i] for i in indel_list]
    
    # Remove indels, in reverse order to maintain indices,
    # from temp_seq_list
    for index in sorted(indel_list, reverse=True):
        del temp_prot1_list[index]
        del temp_prot2_list[index]
        
    # Set the reduced protein sequence lists, indel list, and insert list
    # as elements in the data dictionary.
    # NOT SURE WHAT THE STRUCTURE OF THIS DICTIONARY SHOULD BE
    data['protein1']['reduced_list'] = temp_prot1_list
    data['protein2']['reduced_list'] = temp_prot2_list
    data['diff_dict']['indel_index'] = indel_list
    data['diff_dict']['indel_new_aa'] = insert_list

In [301]:
find_indels(data)

# Finding Differences
<a href="#TOC">TOC</a>

In [303]:
def find_differences(data):
    """
    Find the differences between two clustal aligned protein sequences.
    
    Parameters
    ----------
    data: dict
        A nested dictionary defined by the isor_primer program.
        
    Returns
    -------
    Nothing. This populates the diff_dict in that larger supplied data dictionary.
    
    TODO
    ----
    MAKE SURE THIS FUNCTION CALL COMES AFTER THE FIND_INDELS CALL.
    """
    
    # I SHOULD PUT THIS AS A GENERAL CHECK AT THE BEGINNING OF MY SCRIPT
    """try:
        assert len(prot1) > 0 and len(prot2) > 0
    except:
        raise Exception("Make sure you have two protein sequences loaded.")"""

    # USE THE LIST VERSION OF THE PROTEIN SEQUENCE
    # LIST INDEXING IS FASTER
    prot1 = data['protein1']['reduced_list']
    prot2 = data['protein2']['reduced_list']
        
    # Comparing the two sequences pairwise
    for i,res in enumerate(prot1):
        if prot1[i] != prot2[i]:
            data['diff_dict']['swap_index'].append(i)
            data['diff_dict']['swap_old_aa'].append(prot1[i])
            data['diff_dict']['swap_new_aa'].append(prot2[i])

In [304]:
find_differences(data)

# Codon Swap
<a href="#TOC">TOC</a>
Do the codon swapping separate from finding indels and differences.

In [310]:
def codon_swap(data):
    """
    Grabs new codons given a one-letter amino acid code.
    
    Parameters
    ----------
    data: dict
        A nested dictionary defined by the isor_primer program.
        
    Returns
    -------
    Nothing. This adds to the supplied data dictionary.
    
    TODO
    ----
    MAKE SURE THIS FUNCTION CALL COMES AFTER THE FIND_INDELS CALL.
    """
    
    # The back translation table for NNS codon with human preference
    back_table = {
        'A':'GCC', 
        'C':'TGC', 
        'D':'GAC', 
        'E':'GAG', 
        'F':'TTC', 
        'G':'GGC', 
        'H':'CAC', 
        'I':'ATC', 
        'K':'AAG', 
        'L':'CTG', 
        'M':'ATG', 
        'N':'AAC', 
        'P':'CCC', 
        'Q':'CAG', 
        'R':'CGG', 
        'S':'AGC', 
        'T':'ACC', 
        'V':'GTG', 
        'W':'TGG', 
        'Y':'TAC',
        '-':''
       }
    
    # New codons for indel insertion
    data['diff_dict']['indel_new_codon'] = [back_table[res] for res in data['diff_dict']['indel_new_aa']]
    # New codons for amino acid swaps
    data['diff_dict']['swap_new_codon'] = [back_table[res] for res in data['diff_dict']['swap_new_aa']]


In [311]:
codon_swap(data)

# Build Primers
<a href="#TOC">TOC</a>

In [364]:
def build_primers(data):
    """
    Pulls 5 codons upstream and downstream of the mutation site and builds 
    a 30 or 33 base pair primer.
    
    Populates the data['primers'] nested dictionary
    
    TODO
    ----
    Deal with the issue of needing more sequence upstream and downstream of the
    protein coding sequence.
    """

    primers = {'indel':[], 'swap':[]}

    if len(data['diff_dict']['indel_index']) > 0:
        for i in data['diff_dict']['indel_index']:
            upstream = data['dna1']['codon_list'][(i-1)-5:(i-1)]
            downstream = data['dna1']['codon_list'][i:i+5]
            primers['indel'].append(''.join(upstream + downstream))
            
    if len(data['diff_dict']['swap_index']) > 0:
        for n,site in enumerate(data['diff_dict']['swap_index']):
            # Pull the 5 codons upstream of the swap site from the codon list
            upstream = data['dna1']['codon_list'][(site-1)-5:(site-1)]
            # Pull the 5 codons downstream of the swap site from the codon list
            downstream = data['dna1']['codon_list'][(site+1):(site+1)+5]
            # Add the upstream codons + the new site codon + the downstream codons
            primers['swap'].append(''.join(upstream + list(data['diff_dict']['swap_new_codon'][n]) + downstream))
            
    data['primers'] = primers

In [365]:
build_primers(data)

# REALIZATIONS
<a href="#TOC">TOC</a>

I need logic to deal with insertions larger than one amino acid.

# Replacement Codons
Make a preferred codon dictionary. Could add some logic to check for stop codons. Maybe a first, second, and third choice set of codon dictionaries?

<a href="#TOC">TOC</a>

In [186]:
print(CodonTable.unambiguous_dna_by_id[1])

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

## Table Generation
<a href=#TOC>TOC</a>

In [75]:
# Building an NNS codon dictionary
# http://lists.open-bio.org/pipermail/biopython/2007-August/003660.html

t = CodonTable.standard_dna_table

bt = dict()
for a1 in "ATCG" :
    for a2 in "ATCG" :
        for a3 in "CG" :
            codon = a1+a2+a3
            try :
                amino = t.forward_table[codon]
            except KeyError :
                assert codon in t.stop_codons
                continue
            try:
                bt[amino].append(codon)
            except KeyError :
                bt[amino] = [codon]

for amino in sorted(bt.keys()) :
     print("'{}':{}, ".format(amino, bt[amino]))


'A':['GCC', 'GCG'], 
'C':['TGC'], 
'D':['GAC'], 
'E':['GAG'], 
'F':['TTC'], 
'G':['GGC', 'GGG'], 
'H':['CAC'], 
'I':['ATC'], 
'K':['AAG'], 
'L':['TTG', 'CTC', 'CTG'], 
'M':['ATG'], 
'N':['AAC'], 
'P':['CCC', 'CCG'], 
'Q':['CAG'], 
'R':['AGG', 'CGC', 'CGG'], 
'S':['AGC', 'TCC', 'TCG'], 
'T':['ACC', 'ACG'], 
'V':['GTC', 'GTG'], 
'W':['TGG'], 
'Y':['TAC'], 


In [72]:
# Choosing the codon with the highest usage in humans
# https://www.genscript.com/tools/codon-frequency-table
back_table = {
        'A':'GCC', 
        'C':'TGC', 
        'D':'GAC', 
        'E':'GAG', 
        'F':'TTC', 
        'G':'GGC', 
        'H':'CAC', 
        'I':'ATC', 
        'K':'AAG', 
        'L':'CTG', 
        'M':'ATG', 
        'N':'AAC', 
        'P':'CCC', 
        'Q':'CAG', 
        'R':'CGG', 
        'S':'AGC', 
        'T':'ACC', 
        'V':'GTG', 
        'W':'TGG', 
        'Y':'TAC'
       }